In [8]:
import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [9]:
SCOPES = ['https://www.googleapis.com/auth/analytics']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
VIEW_ID = 'ga:170298138'

In [10]:
def initialize_analyticsreporting():
  
  # Parse command-line arguments.
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        parents=[tools.argparser])
    flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
    flow = client.flow_from_clientsecrets(
        CLIENT_SECRETS_PATH, scope=SCOPES,
        message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
    storage = file.Storage('analyticsreporting.dat')
    credentials = storage.get()
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
    analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

    return analytics

In [11]:
def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
    return analytics.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions':[{'name':'ga:acquisitionSource'}], 
          'dimensions':[{'name' : 'ga:landingPagePath'}]
#           'dimensions':[{'name':'ga:acquisitionSource'}],
            
        }]
      }
  ).execute()

In [12]:
def print_response(response):
  
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

    for row in rows:
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        for header, dimension in zip(dimensionHeaders, dimensions):
            print(header + ': ' + dimension)

        for i, values in enumerate(dateRangeValues):
            print('Date range (' + str(i) + ')')
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                print(metricHeader.get('name') + ': ' + value)

In [13]:
def main():
    analytics = initialize_analyticsreporting()
    response = get_report(analytics)
    print(response)
    print()
#     print(response)
    print_response(response)

In [14]:
if __name__ == '__main__':
    main()

{'reports': [{'columnHeader': {'dimensions': ['ga:landingPagePath'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}]}}, 'data': {'rows': [{'dimensions': ['/'], 'metrics': [{'values': ['37']}]}, {'dimensions': ['/2018/02/17/cloud-server/'], 'metrics': [{'values': ['2']}]}, {'dimensions': ['/2018/02/18/linux_command/'], 'metrics': [{'values': ['1']}]}, {'dimensions': ['/2018/04/09/Flask-with-model/'], 'metrics': [{'values': ['1']}]}, {'dimensions': ['/about/'], 'metrics': [{'values': ['4']}]}, {'dimensions': ['/google7bd9c6e0b4b8c3c7.html'], 'metrics': [{'values': ['1']}]}, {'dimensions': ['/journal/Flask.html'], 'metrics': [{'values': ['1']}]}, {'dimensions': ['/tag/2-computer-science-1/'], 'metrics': [{'values': ['5']}]}, {'dimensions': ['/tag/4-machine-learning-1/'], 'metrics': [{'values': ['1']}]}, {'dimensions': ['/tag/6-project/'], 'metrics': [{'values': ['1']}]}], 'totals': [{'values': ['54']}], 'rowCount': 10, 'minimums': [{'values': ['1']}], 